In [ ]:
# import some packages
import random
import os
import xarray as xr
import numpy as np
import xgcm # this is an xarray-based package to deal with GCM output
from matplotlib import pyplot as plt
import cmocean # this is a colormap package
import cartopy.crs as ccrs # this is a mapping package
import gsw
from scipy.ndimage import convolve
from scipy.ndimage import gaussian_filter as gf
%matplotlib inline
import matplotlib as mpl 
import seaborn as sns
sns.set(style='ticks', context='notebook',font_scale=1.3)
mpl.rc('figure', dpi=100)
mpl.rc('savefig', dpi=500, bbox='tight')
mpl.rc('legend', frameon=False)

plt.rcParams['font.size'] = 16

## Making a method for ploting the Eddies and Kurishio,and then making analysis based on the plots.

1. plot the existing variables to see them roughly
2. Calculate all of the variables we need into the dataset 
3. Make a rough plot for these new variables
4. Pick a eddy that is related with Kuroshio 
5. Ploting variables to see how the eddy looks like in every plot
6. See the eddy within time steps 
7. download these pictures then make a powerpoint

Local data from NASA

In [ ]:
ds_time = xr.open_dataset('taiwan_snapshot_30timesteps.nc')

In [ ]:
ds = ds_time.isel(time = 1) ## making the defult to be the first time in the timesteps dataset

In [ ]:
grid = xgcm.Grid(ds, periodic=False)
grid #slicing the plot and data into XY grids

# Sea Surface Height

In [ ]:
# same for SST and SSS
#Below notice that Eta, SST, SSS are at (i,j) indices dimensions, 
#so at the center of grid cell, but Zeta is at (i_g,j_g), so at the sides of grid cells
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.SSH.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Sea surface height');

## Sea Surface Temperature

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.SST.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Sea surface Temperature');

## Vorticity

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity')

## Sea Surface Salinity

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.SSS.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Sea surface Salinity');

### Next: Calculate Variables

In [ ]:
ds['eta_prime'] = ds.SSH - ds.SSH.mean() # removing the mean

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))

ds.eta_prime.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)
ax.set_title('Eta_prime')

#### Gridiant Eta Sea Surface Height

dEta/dx and dEta/dy, at center

In [ ]:
ds['eta_filt'] =(('j','i'), gf(ds.SSH,5,mode = 'mirror')) #gaussian filter, other than remove the mean 

In [ ]:
metrics = {
 ('X',): ['dxC', 'dxG'], # X distances
 ('Y',): ['dyC', 'dyG'], # Y distances
# ('Z',): ['drW', 'drS', 'drC'], # Z distances
 ('X', 'Y'): ['rA', 'rAz', 'rAs', 'rAw'] # Areas
}

grid = xgcm.Grid(ds, periodic=False, metrics=metrics)

ds['eta_dx'] = grid.derivative(ds.eta_filt, 'X', boundary='extend')
ds['eta_dy'] = grid.derivative(ds.eta_filt, 'Y', boundary='extend')

In [ ]:
ds['eta_dx_abs'] = np.sqrt(grid.interp(ds.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value

In [ ]:
fig, ax = plt.subplots(1,4, figsize = (25,12), sharey = True)

ds.eta_filt.plot(robust = True, ax = ax[0])
ds.eta_dx.plot(robust = True, ax = ax[1])
ds.eta_dy.plot(robust = True, ax = ax[2])
ds.eta_dx_abs.plot(robust = True, ax = ax[3])
ax[0].set_title(r'Sea surface Height after Filtering')
ax[1].set_title(r'Gradiant Eta dx')
ax[2].set_title(r'Gradiant Eta dy')
ax[3].set_title(r'absolute gradiant Eta')

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Absolute Gradiant Eta');

## Geographic velocity

In [ ]:
ds['f'] = gsw.f(ds.YC.mean(dim = 'i')) #Coriolis force

In [ ]:
ds['geo_V'] = 9.8 * ds.eta_dx/ds.f.mean()
ds['geo_U'] = -9.8 * ds.eta_dy/ds.f.mean()

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,10), sharey = True)
ds.geo_V.plot(robust = True, ax = ax[0])
ds.geo_U.plot(robust = True, ax = ax[1])
ax[0].set_title(r'Geographic V')
ax[1].set_title(r'Geographic U')

#### Sea Surface Density

In [ ]:
SA = ds.SSS
CT = ds.SST 
ds["density"] = gsw.rho(SA, CT, 0)-1000

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.density.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Sea surface Density');

#### S_s and S_u

In [ ]:
## U(j, i_g)
## V(j_g, i)
## S_u = (dv/dy + du/dx)^2 
## S_s = (du/dx - dv/dy)^2
## ds['S_u'] = ((ds.V * ds.dyC + ds.U * ds.dxC)**2)/ds.rAz
## ds['S_s'] = ((ds.U * ds.dxC - ds.V * ds.dyC)**2)/ds.rA

ds['SS'] = (grid.diff(ds.U * ds.dxC, 'X', boundary='extend') -
             grid.diff(ds.V * ds.dyC, 'Y', boundary='extend'))/ds.rA
ds['SU'] = (grid.diff(ds.U * ds.dxC, 'Y', boundary='extend') +
             grid.diff(ds.V * ds.dyC, 'X', boundary='extend'))/ds.rAz
ds['zeta'] = (-grid.diff(ds.U * ds.dxC, 'Y', boundary='extend') +
                    grid.diff(ds.V * ds.dyC, 'X', boundary='extend'))/ds.rAz

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.SS.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Strain rate tangentially');

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.SU.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Strain rate normal');

### Okubo force based on the Ss and Su

In [ ]:
## next is to calculatethe W which is ss^2 + su^2 - zeta^2(voticity), W is Okubo Weiss Criterion 
ds['W'] = (grid.interp(ds.SS**2, ['X','Y'], boundary = 'extend')) + (ds.SU)**2 - (ds.zeta)**2

In [ ]:
ds['gf_SS'] = (('j', 'i'), gf(ds.SS, sigma = 10, mode='nearest',cval=0))
ds['gf_SU'] = (('j_g', 'i_g'), gf(ds.SU, sigma = 10, mode='nearest',cval=0))
ds['gf_Zeta'] = (('j_g', 'i_g'), gf(ds.zeta, sigma = 10, mode='nearest',cval=0))

In [ ]:
ds['W_filt'] = (grid.interp(ds.gf_SS**2, ['X','Y'], boundary = 'extend')) + (ds.gf_SU)**2 - (ds.gf_Zeta)**2

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.W.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Okubo Weiss Criterion');

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.W_filt.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Okubo Weiss Criterion after Filter');

#### Rossby Number
vorticity / Coriolis force

In [ ]:
ds['Rossby'] = grid.interp(ds.zeta, ['X','Y'], boundary = 'extend')/ds.f

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.Rossby.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Rossby Number'); ## rossby number related to vorticity

In [ ]:
ds['guassian_filter_Rossby'] = (('j_g', 'i_g'), gf(ds.Rossby, sigma = 10, mode='nearest',cval=0))

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))

ds.guassian_filter_Rossby.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)
ax.set_title('Rossby after Gaussian')

#### Pick one eddy 

Roughly look at these plots above, I would think that vorticity plot show a better image for identify eddies I want. 

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)
ax.set_title(r'Vorticity')

In [ ]:
ds2 = ds_time.isel(time = 2)

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds2.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 2')

In [ ]:
ds2['SS'] = (grid.diff(ds2.U * ds2.dxC, 'X', boundary='extend') -
             grid.diff(ds2.V * ds2.dyC, 'Y', boundary='extend'))/ds2.rA
ds2['SU'] = (grid.diff(ds2.U * ds2.dxC, 'Y', boundary='extend') +
             grid.diff(ds2.V * ds2.dyC, 'X', boundary='extend'))/ds2.rAz
ds2['gf_SS'] = (('j', 'i'), gf(ds2.SS, sigma = 10, mode='nearest',cval=0))
ds2['gf_SU'] = (('j_g', 'i_g'), gf(ds2.SU, sigma = 10, mode='nearest',cval=0))
ds2['gf_Zeta'] = (('j_g', 'i_g'), gf(ds2.zeta, sigma = 10, mode='nearest',cval=0))
ds2['W_filt'] = (grid.interp(ds2.gf_SS**2, ['X','Y'], boundary = 'extend')) + (ds2.gf_SU)**2 - (ds2.gf_Zeta)**2

In [ ]:
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds2.W_filt.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(), 
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title('Okubo Weiss Criterion after Filter time 2');

In [ ]:
ds2['eta_filt'] =(('j','i'), gf(ds.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds2['eta_dx'] = grid.derivative(ds2.eta_filt, 'X', boundary='extend')
ds2['eta_dy'] = grid.derivative(ds2.eta_filt, 'Y', boundary='extend')
ds2['eta_dx_abs'] = np.sqrt(grid.interp(ds2.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds2.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value

In [ ]:
central_lon = ds2.XC.mean().values
central_lat = ds2.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds2.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 2')

In [ ]:
ds3 = ds_time.isel(time = 3)
central_lon = ds3.XC.mean().values
central_lat = ds3.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds3.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 3')
plt.savefig('vorticity_time3.png')

In [ ]:
ds4 = ds_time.isel(time = 4)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds4.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 4')

In [ ]:
ds4['eta_filt'] =(('j','i'), gf(ds4.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds4['eta_dx'] = grid.derivative(ds4.eta_filt, 'X', boundary='extend')
ds4['eta_dy'] = grid.derivative(ds4.eta_filt, 'Y', boundary='extend')
ds4['eta_dx_abs'] = np.sqrt(grid.interp(ds4.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds4.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value

In [ ]:
central_lon = ds4.XC.mean().values
central_lat = ds4.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds4.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 4')


In [ ]:
ds5 = ds_time.isel(time = 5)
central_lon = ds5.XC.mean().values
central_lat = ds5.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds5.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 5')

In [ ]:
ds6 = ds_time.isel(time = 6)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds6.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 6')

In [ ]:
ds6['eta_filt'] =(('j','i'), gf(ds6.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds6['eta_dx'] = grid.derivative(ds6.eta_filt, 'X', boundary='extend')
ds6['eta_dy'] = grid.derivative(ds6.eta_filt, 'Y', boundary='extend')
ds6['eta_dx_abs'] = np.sqrt(grid.interp(ds6.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds6.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value

In [ ]:
central_lon = ds6.XC.mean().values
central_lat = ds6.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds6.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 6')


In [ ]:
ds7 = ds_time.isel(time = 7)
central_lon = ds5.XC.mean().values
central_lat = ds5.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds7.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 7')
plt.savefig('vorticity_time7')

In [ ]:
ds8 = ds_time.isel(time = 8)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds8.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 8')

In [ ]:
ds8['eta_filt'] =(('j','i'), gf(ds8.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds8['eta_dx'] = grid.derivative(ds8.eta_filt, 'X', boundary='extend')
ds8['eta_dy'] = grid.derivative(ds8.eta_filt, 'Y', boundary='extend')
ds8['eta_dx_abs'] = np.sqrt(grid.interp(ds8.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds8.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds8.XC.mean().values
central_lat = ds8.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds8.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 8')

In [ ]:
ds9 = ds_time.isel(time = 9)
central_lon = ds5.XC.mean().values
central_lat = ds5.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds9.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 9')
plt.savefig('vorticity_time9')

In [ ]:
ds10 = ds_time.isel(time = 10)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds10.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 10')


In [ ]:
ds10['eta_filt'] =(('j','i'), gf(ds10.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds10['eta_dx'] = grid.derivative(ds10.eta_filt, 'X', boundary='extend')
ds10['eta_dy'] = grid.derivative(ds10.eta_filt, 'Y', boundary='extend')
ds10['eta_dx_abs'] = np.sqrt(grid.interp(ds10.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds10.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds10.XC.mean().values
central_lat = ds10.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds10.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 10')
plt.savefig('eta_abs_time10.png')

In [ ]:
ds11 = ds_time.isel(time = 11)
central_lon = ds5.XC.mean().values
central_lat = ds5.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds11.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 11')
plt.savefig('vorticity_time11')

In [ ]:
ds12 = ds_time.isel(time = 12)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds12.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 12')


In [ ]:
ds12['eta_filt'] =(('j','i'), gf(ds12.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds12['eta_dx'] = grid.derivative(ds12.eta_filt, 'X', boundary='extend')
ds12['eta_dy'] = grid.derivative(ds12.eta_filt, 'Y', boundary='extend')
ds12['eta_dx_abs'] = np.sqrt(grid.interp(ds12.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds12.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds12.XC.mean().values
central_lat = ds12.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds12.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 12')
plt.savefig('eta_abs_time12.png')

In [ ]:
ds13 = ds_time.isel(time = 13)
central_lon = ds5.XC.mean().values
central_lat = ds5.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds13.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 13')
plt.savefig('vorticity_time13')

In [ ]:
ds14 = ds_time.isel(time = 14)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds14.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 14')


In [ ]:
ds14['eta_filt'] =(('j','i'), gf(ds14.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds14['eta_dx'] = grid.derivative(ds14.eta_filt, 'X', boundary='extend')
ds14['eta_dy'] = grid.derivative(ds14.eta_filt, 'Y', boundary='extend')
ds14['eta_dx_abs'] = np.sqrt(grid.interp(ds14.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds14.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds14.XC.mean().values
central_lat = ds14.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds14.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 14')
plt.savefig('eta_abs_time14.png')

In [ ]:
ds15 = ds_time.isel(time = 15)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds15.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 15')
plt.savefig('vorticity_time15')

In [ ]:
ds16 = ds_time.isel(time = 16)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds16.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 16')


In [ ]:
ds16['eta_filt'] =(('j','i'), gf(ds16.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds16['eta_dx'] = grid.derivative(ds16.eta_filt, 'X', boundary='extend')
ds16['eta_dy'] = grid.derivative(ds16.eta_filt, 'Y', boundary='extend')
ds16['eta_dx_abs'] = np.sqrt(grid.interp(ds16.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds16.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds16.XC.mean().values
central_lat = ds16.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds16.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 16')
plt.savefig('eta_abs_time16.png')

In [ ]:
ds17 = ds_time.isel(time = 17)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds17.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 17')
plt.savefig('vorticity_time17')

In [ ]:
ds18 = ds_time.isel(time = 18)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds18.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 18')


In [ ]:
ds18['eta_filt'] =(('j','i'), gf(ds18.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds18['eta_dx'] = grid.derivative(ds18.eta_filt, 'X', boundary='extend')
ds18['eta_dy'] = grid.derivative(ds18.eta_filt, 'Y', boundary='extend')
ds18['eta_dx_abs'] = np.sqrt(grid.interp(ds18.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds18.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds18.XC.mean().values
central_lat = ds18.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds18.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 18')
plt.savefig('eta_abs_time18.png')

In [ ]:
ds19 = ds_time.isel(time = 19)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds19.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 19')
plt.savefig('vorticity_time19')

In [ ]:
ds20 = ds_time.isel(time = 20)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds20.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 20')


In [ ]:
ds20['eta_filt'] =(('j','i'), gf(ds20.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds20['eta_dx'] = grid.derivative(ds20.eta_filt, 'X', boundary='extend')
ds20['eta_dy'] = grid.derivative(ds20.eta_filt, 'Y', boundary='extend')
ds20['eta_dx_abs'] = np.sqrt(grid.interp(ds20.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds20.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds20.XC.mean().values
central_lat = ds20.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds20.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 20')
plt.savefig('eta_abs_time20.png')

In [ ]:
ds21 = ds_time.isel(time = 21)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds21.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 21')
plt.savefig('vorticity_time21')

In [ ]:
ds22 = ds_time.isel(time = 22)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds22.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 22')

In [ ]:
ds22['eta_filt'] =(('j','i'), gf(ds22.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds22['eta_dx'] = grid.derivative(ds22.eta_filt, 'X', boundary='extend')
ds22['eta_dy'] = grid.derivative(ds22.eta_filt, 'Y', boundary='extend')
ds22['eta_dx_abs'] = np.sqrt(grid.interp(ds22.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds22.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds22.XC.mean().values
central_lat = ds22.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds22.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 22')
plt.savefig('eta_abs_time22.png')

In [ ]:
ds23 = ds_time.isel(time = 23)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds23.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 23')
plt.savefig('vorticity_time23')

In [ ]:
ds24 = ds_time.isel(time = 24)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds24.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 24')

In [ ]:
ds24['eta_filt'] =(('j','i'), gf(ds24.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds24['eta_dx'] = grid.derivative(ds24.eta_filt, 'X', boundary='extend')
ds24['eta_dy'] = grid.derivative(ds24.eta_filt, 'Y', boundary='extend')
ds24['eta_dx_abs'] = np.sqrt(grid.interp(ds24.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds24.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds24.XC.mean().values
central_lat = ds24.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds24.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 24')
plt.savefig('eta_abs_time24.png')

In [ ]:
ds25 = ds_time.isel(time = 25)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds25.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 25')
plt.savefig('vorticity_time25')

In [ ]:
ds26 = ds_time.isel(time = 26)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds26.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 26')

In [ ]:
ds26['eta_filt'] =(('j','i'), gf(ds26.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds26['eta_dx'] = grid.derivative(ds26.eta_filt, 'X', boundary='extend')
ds26['eta_dy'] = grid.derivative(ds26.eta_filt, 'Y', boundary='extend')
ds26['eta_dx_abs'] = np.sqrt(grid.interp(ds26.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds26.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds26.XC.mean().values
central_lat = ds26.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds26.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 26')
plt.savefig('eta_abs_time26.png')

In [ ]:
ds27 = ds_time.isel(time = 27)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds27.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 27')
plt.savefig('vorticity_time27')

In [ ]:
ds28 = ds_time.isel(time = 28)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds28.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 28')

In [ ]:
ds28['eta_filt'] =(('j','i'), gf(ds28.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds28['eta_dx'] = grid.derivative(ds28.eta_filt, 'X', boundary='extend')
ds28['eta_dy'] = grid.derivative(ds28.eta_filt, 'Y', boundary='extend')
ds28['eta_dx_abs'] = np.sqrt(grid.interp(ds28.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds28.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds28.XC.mean().values
central_lat = ds28.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds28.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 28')
plt.savefig('eta_abs_time28.png')

In [ ]:
ds29 = ds_time.isel(time = 29)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds29.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 29')
plt.savefig('vorticity_time29')

In [ ]:
ds30 = ds_time.isel(time = 30)
central_lon = ds.XC.mean().values
central_lat = ds.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds30.zeta.plot(ax=ax, x='XG', y='YG',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Vorticity time 30')

In [ ]:
ds30['eta_filt'] =(('j','i'), gf(ds30.SSH,5,mode = 'nearest'))
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
ds30['eta_dx'] = grid.derivative(ds30.eta_filt, 'X', boundary='extend')
ds30['eta_dy'] = grid.derivative(ds30.eta_filt, 'Y', boundary='extend')
ds30['eta_dx_abs'] = np.sqrt(grid.interp(ds30.eta_dx**2, 'X', boundary = 'extend') + 
                           grid.interp(ds30.eta_dy**2,'Y', boundary = 'extend')) # eta dx absolute value
central_lon = ds30.XC.mean().values
central_lat = ds30.YC.mean().values

fig = plt.figure(figsize=(13, 10))
ax = fig.add_axes([0, 0.02, 1, 0.91],
                  projection=ccrs.Orthographic(central_lon, central_lat))
ds30.eta_dx_abs.plot(ax=ax, x='XC', y='YC',  robust=True, transform=ccrs.PlateCarree(),
            cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)

ax.set_title(r'Gradiant Absolute time 30')
plt.savefig('eta_abs_time30.png')

In [ ]:
ffmpeg -framerate 10 -pattern_type glob -i '*.png' -c:v libx264 -pix_fmt yuv420p out.mp4